<a href="https://colab.research.google.com/github/Vyoma-garg/Natural-Language-Processing/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Getting dataset**

In [ ]:
import pandas as pd
import gzip
false=False
true=True

def parse(path):
  g = gzip.open('/content/Magazine_Subscriptions.json.gz', 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions.json.gz')

Total Number of Magazine Subscriptions reviews 

In [ ]:
len(df)

89689

Total Number of reviews taken in consideration 

In [ ]:
df=df[:25000]
len(df)

25000

In [ ]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN


Adding a column for Positive and Negative Sentiment

In [ ]:
df.loc[df['overall']<=3, 'Sentiment']='Negative'
df.loc[df['overall']>3, 'Sentiment']='Positive'
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive


Adding a column for Sentiments in binary form

In [ ]:
df.loc[df['Sentiment']=='Negative', 'Binary_Sentiment']=0
df.loc[df['Sentiment']=='Positive', 'Binary_Sentiment']=1
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive,1.0
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive,1.0
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative,0.0
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive,1.0


In [ ]:
df.tail()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
24995,5.0,NaN,True,"03 24, 2015",A1AG7VJ1T35X77,B00005QJE0,Darrin R. West,Gorgeous large format magazine. Very inspiring...,Gorgeous and Inspiring.,1427155200,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24996,4.0,NaN,True,"07 30, 2014",A365DG6L5EUQQB,B00005QJE0,Charles Belew,Very happy with our subscribtions!,Four Stars,1406678400,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24997,5.0,NaN,True,"02 26, 2014",A312FAW297PIOZ,B00005QJE0,kwilson,Love advice on gardening and the different gar...,Would. recommend this magazine to everyone .,1393372800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24998,5.0,NaN,True,"04 4, 2013",AUHOX5LLRIF11,B00005QJE0,Amazon Customer,photos are so good I look at these mags over a...,best gardening magazine,1365033600,{'Format:': ' Print Magazine'},NaN,Positive,1.0
24999,5.0,4,True,"02 6, 2013",A2P4CPM9S3QBDU,B00005QJE0,Linda W Owen,The English Garden is a wonderful gardening ma...,"The English Garden, good magizine for Americans",1360108800,{'Format:': ' Print Magazine'},NaN,Positive,1.0


## **Getting the corpus and binary sentiments**

In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopset=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
corpus = df["reviewText"].fillna(' ').values.tolist()
rating=df['overall'].values.tolist()
sentiment=df['Sentiment'].values.tolist()
bin_sentiment=df['Binary_Sentiment'].values.tolist()
print(bin_sentiment)

## **WORD EMBEDDINGS**

### **(A) TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#create transform
vectorizer2=TfidfVectorizer(use_idf=True, lowercase=True, stop_words=stopset)

#tokenize and build vocab
X_tfidf=vectorizer2.fit_transform(corpus) 
print(X_tfidf.shape)

#printing idf values
print(vectorizer2.idf_)

df5 = pd.DataFrame(X_tfidf[0].T.todense(), index=vectorizer2.get_feature_names(), columns=["TF-IDF"])
df5 = df5.sort_values('TF-IDF', ascending=False)
print (df5.head(25))

(25000, 25711)
[ 6.0768151   6.85000498 10.43352392 ... 10.43352392 10.43352392
 10.43352392]
               TF-IDF
maxpc        0.373384
boot         0.236606
computing    0.222767
enthusiast   0.208770
cd           0.190191
obcessed     0.145426
divx         0.145426
savorying    0.145426
codecs       0.145426
encoding     0.139774
merge        0.135764
icing        0.132654
broadband    0.130113
rom          0.126103
blessed      0.124461
utilities    0.122993
formerly     0.121664
connections  0.120452
bang         0.119336
audio        0.118303
cake         0.116442
sight        0.112651
profit       0.109149
welcome      0.109149
buck         0.107223


### **(B) Word2Vec**


Getting the training and test dataset

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
#X_data, y_data = corpus, bin_sentiment
X_data, y_data = np.array(corpus), np.array(bin_sentiment)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state = 40)
print('Data Split done.')
X_data.shape
y_data.shape

y_test.shape
type(X_train)
X_train.shape

Data Split done.


(18750,)

In [ ]:
from gensim.models import Word2Vec, KeyedVectors
import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from gensim.models import Word2Vec

Embedding_dimensions = 100

# Creating Word2Vec training dataset.
Word2vec_train_data = list(map(lambda x: x.split(), X_train))
len(Word2vec_train_data)

18750

In [ ]:
# Defining the model and training it.
word2vec_model = Word2Vec(Word2vec_train_data,
                 size=Embedding_dimensions,
                 workers=8,
                 min_count=5)

print("Vocabulary Length:", len(word2vec_model.wv.vocab))

Vocabulary Length: 9640


In [ ]:
input_length = 60

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer(filters="", lower=True)
tokenizer.fit_on_texts(corpus)

vocab_length = len(tokenizer.word_index) + 1
print("Tokenizer vocab length:", vocab_length)
lili=list(tokenizer.word_index)
#lili


Tokenizer vocab length: 64615


In [ ]:
tokenizer.word_index.items()

In [ ]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=input_length)
X_test1  = pad_sequences(tokenizer.texts_to_sequences(X_test) , maxlen=input_length)

print("X_train.shape:", X_train.shape)
print("X_test.shape :", X_test1.shape)
X_test1

WORD2VEC EMBEDDING MATRIX

In [ ]:
import numpy as np
embedding_matrix = np.zeros((vocab_length, Embedding_dimensions))

for word, token in tokenizer.word_index.items():
    if word2vec_model.wv.__contains__(word):
        embedding_matrix[token] = word2vec_model.wv.__getitem__(word)

print("Embedding Matrix Shape:", embedding_matrix.shape)
embedding_matrix


### **(C) GLOVE embeddings**

In [ ]:
!wget https://nlp.stanford.edu/data/glove.6B.zip

--2021-05-02 07:38:15--  https://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-05-02 07:38:15--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6B.zip        100%[===================>] 822.24M  5.29MB/s    in 2m 40s  

utime(glove.6B.zip): No such file or directory
2021-05-02 07:40:56 (5.13 MB/s) - ‘glove.6B.zip’ saved [862182613/862182613]



In [ ]:
!unzip glove*.zip

unzip:  cannot find or open glove*.zip, glove*.zip.zip or glove*.zip.ZIP.

No zipfiles found.


In [ ]:
embeddings_index = dict()

f = open('/content/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

GLOVE EMBEDDING MATRIX

In [ ]:
embedding_matrix_glove = np.zeros((vocab_length,100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_glove[i] = embedding_vector
embedding_matrix_glove.shape
embedding_matrix_glove


## **Sentiment Analysis Using following models**

## **Q3 (a)BiLSTM**

### **(i)BiLSTM Using Word2Vec embedding**

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, GlobalMaxPool1D, Dense, LSTM, Conv1D, Embedding

In [ ]:
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length, 
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix], 
                                input_length=input_length,
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'), cat [0.1,01]
                                   f      dog [0.2]
    ],
    name="Sentiment_Model")
    return model


In [ ]:
training_model = getModel()
training_model.summary()

Model: "Sentiment_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 100)           6461500   
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 200)           160800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 200)           240800    
_________________________________________________________________
conv1d (Conv1D)              (None, 56, 100)           100100    
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                1616      
_________________________________________________________________
dense_1 (Dense)              (None, 1)             

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = training_model.fit(
    X_train, y_train,
    batch_size=1024,
    epochs=5,
    validation_split=0.1,
    #callbacks=callbacks,
    #verbose=1,
)

Epoch 1/5
17/17 [==============================] - 108s 6s/step - loss: 0.5729 - accuracy: 0.7086 - val_loss: 0.4083 - val_accuracy: 0.7888
Epoch 2/5
17/17 [==============================] - 97s 6s/step - loss: 0.4146 - accuracy: 0.7965 - val_loss: 0.3782 - val_accuracy: 0.8149
Epoch 3/5
17/17 [==============================] - 95s 6s/step - loss: 0.3821 - accuracy: 0.8187 - val_loss: 0.3703 - val_accuracy: 0.8235
Epoch 4/5
17/17 [==============================] - 96s 6s/step - loss: 0.3660 - accuracy: 0.8308 - val_loss: 0.3596 - val_accuracy: 0.8347
Epoch 5/5
17/17 [==============================] - 96s 6s/step - loss: 0.3627 - accuracy: 0.8325 - val_loss: 0.3548 - val_accuracy: 0.8379


Test results

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix
print(X_test1.shape)     
y_pred = training_model.predict(X_test1)
y_pred = np.where(y_pred>=0.5, 1, 0)
print(y_pred)
print(y_pred.shape)   # train : 18750 x60 \\\\ test : 6250x60\\\\\ ypred 
print(type(y_pred))

In [ ]:
print(y_test)
print(y_test.shape)
y_test1=y_test.reshape((6250,1))
y_test1.shape

In [ ]:
print(accuracy_score(y_test1,y_pred))
print(f1_score(y_test1,y_pred))
print(confusion_matrix(y_test1,y_pred))

0.8368
0.886968085106383
[[1228  520]
 [ 500 4002]]


### **(ii)BiLSTM using Glove Embeddings**

In [ ]:
vocab_length=64615

In [ ]:
def getModel():
    embedding_layer = Embedding(input_dim = vocab_length, 
                                output_dim = Embedding_dimensions,
                                weights=[embedding_matrix_glove], 
                                input_length=input_length, #60 
                                trainable=False)

    model = Sequential([
        embedding_layer,
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Bidirectional(LSTM(100, dropout=0.3, return_sequences=True)),
        Conv1D(100, 5, activation='relu'),
        GlobalMaxPool1D(),
        Dense(16, activation='relu'),
        Dense(1, activation='sigmoid'),
    ],
    name="Sentiment_Model")
    return model


In [ ]:
training_model = getModel()
training_model.summary()

Model: "Sentiment_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 60, 100)           6461500   
_________________________________________________________________
bidirectional (Bidirectional (None, 60, 200)           160800    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 60, 200)           240800    
_________________________________________________________________
conv1d (Conv1D)              (None, 56, 100)           100100    
_________________________________________________________________
global_max_pooling1d (Global (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                1616      
_________________________________________________________________
dense_1 (Dense)              (None, 1)             

In [ ]:
training_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = training_model.fit(
    X_train, y_train,
    batch_size=1024,
    epochs=5,
    validation_split=0.1,
    #callbacks=callbacks,
    #verbose=1,
)

Epoch 1/5
17/17 [==============================] - 102s 5s/step - loss: 0.6216 - accuracy: 0.6628 - val_loss: 0.4881 - val_accuracy: 0.7648
Epoch 2/5
17/17 [==============================] - 92s 5s/step - loss: 0.4861 - accuracy: 0.7573 - val_loss: 0.4276 - val_accuracy: 0.7888
Epoch 3/5
17/17 [==============================] - 91s 5s/step - loss: 0.4244 - accuracy: 0.7968 - val_loss: 0.4036 - val_accuracy: 0.8080
Epoch 4/5
17/17 [==============================] - 91s 5s/step - loss: 0.4062 - accuracy: 0.8056 - val_loss: 0.3982 - val_accuracy: 0.8101
Epoch 5/5
17/17 [==============================] - 91s 5s/step - loss: 0.3993 - accuracy: 0.8101 - val_loss: 0.3864 - val_accuracy: 0.8245


Test Results

In [ ]:
#X_data, y_data = corpus, bin_sentiment
X_data, y_data = np.array(corpus), np.array(bin_sentiment)

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, random_state = 40)
print('Data Split done.')

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix
print(X_test1.shape)     
y_pred = training_model.predict(X_test1)
y_pred = np.where(y_pred>=0.5, 1, 0)
print(y_pred)
print(y_pred.shape)   # train : 18750 x60 \\\\ test : 6250x60\\\\\ ypred 
print(type(y_pred))

In [ ]:
print(y_test)
print(y_test.shape)
y_test1=y_test.reshape((6250,1))
y_test1.shape

In [ ]:
print(accuracy_score(y_test1,y_pred))
print(f1_score(y_test1,y_pred))
print(confusion_matrix(y_test1,y_pred)) 

0.82592
0.8799646954986762
[[1174  574]
 [ 514 3988]]


## **Q3(b) BERT based Sentiment analysis**

In [ ]:
df11=df[['reviewText','Binary_Sentiment']]

In [ ]:

# Dropping the columns having NaN/NaT values
df12= df11.dropna()
  
# Resetting the indices using df.reset_index()
df12 = df12.reset_index(drop=True)
  
df12

,reviewText,Binary_Sentiment
0,"for computer enthusiast, MaxPC is a welcome si...",1.0
1,Thank god this is not a Ziff Davis publication...,1.0
2,Antiques Magazine is a publication made for an...,0.0
3,This beautiful magazine is in itself a work of...,1.0
4,A great read every issue.,1.0
...,...,...
24986,Gorgeous large format magazine. Very inspiring...,1.0
24987,Very happy with our subscribtions!,1.0
24988,Love advice on gardening and the different gar...,1.0
24989,photos are so good I look at these mags over a...,1.0


In [ ]:
from sklearn.model_selection import train_test_split
train,test= train_test_split(df12,test_size=0.2,random_state = 1)

In [ ]:
!pip install transformers

     |████████████████████████████████| 2.3MB 4.3MB/s 
     |████████████████████████████████| 901kB 28.8MB/s 
     |████████████████████████████████| 3.3MB 33.6MB/s 


In [ ]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf

In [ ]:
def convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[DATA_COLUMN], 
                                                          text_b = None,
                                                          label = x[LABEL_COLUMN]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

  train_InputExamples, validation_InputExamples = convert_data_to_examples(train, 
                                                                           test, 
                                                                           'DATA_COLUMN', 
                                                                           'LABEL_COLUMN')
    
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )


DATA_COLUMN = 'reviewText'
LABEL_COLUMN = 'Binary_Sentiment'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train, test, DATA_COLUMN, LABEL_COLUMN)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2110: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

model.fit(train_data, epochs=2, validation_data=validation_data)

Epoch 1/2
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autogr

## **Q3(d)Traditional ML models**

###**(i)Naive Bayes**

In [ ]:
import pandas as pd
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix

**Naive bayes Using TF-IDF**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf1=naive_bayes.MultinomialNB()
clf1.fit(X_train,Y_train)
yp=clf1.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Naive Bayes Using Word2Vec**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix[:25000]),bin_sentiment,random_state=40)  #embed:64615x100... bin_senti: 25000,1
print(X_test.shape)
print(X_test[:,1].shape)
clf1=naive_bayes.MultinomialNB()
clf1.fit(X_train,Y_train)
yp=clf1.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Naive Bayes using Glove embeddings**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix_glove[:25000]),bin_sentiment,random_state=40)  #embed:64615x100... bin_senti: 25000,1
print(X_test.shape)
print(X_test[:,1].shape)
clf1=naive_bayes.MultinomialNB()
clf1.fit(X_train,Y_train)
yp=clf1.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

###**(ii)Decision Tree**

In [ ]:
from sklearn import tree

**Decision Tree Using TF-IDF**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40) #25% test data and 75% train data

clf3=tree.DecisionTreeClassifier()
clf3.fit(X_train,Y_train)
yp=clf3.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Decision Tree with Word2Vec Embedding**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix[:25000]),bin_sentiment,random_state=40) #25% test data and 75% train data

clf3=tree.DecisionTreeClassifier()
clf3.fit(X_train,Y_train)
yp=clf3.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Decision Tree With Glove embeddings**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix_glove[:25000]),bin_sentiment,random_state=40) #25% test data and 75% train data

clf3=tree.DecisionTreeClassifier()
clf3.fit(X_train,Y_train)
yp=clf3.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

### **(iii)Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

**Logistic Regression Using TF-IDF**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf2=LogisticRegression()
clf2.fit(X_train,Y_train)
yp=clf2.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Logisitic Regression Using Word2Vec**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix[:25000]),bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf2=LogisticRegression()
clf2.fit(X_train,Y_train)
yp=clf2.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

**Logistic Regression Using Glove embeddings**

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(abs(embedding_matrix_glove[:25000]),bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf2=LogisticRegression()
clf2.fit(X_train,Y_train)
yp=clf2.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))